In [2]:
from ase.io import read
from utils.visualization.visualizator import sith_viewer

# Visualization

Load the configuration and start the visualization. All modifications will appear in the same graph.

In [3]:
atoms = read('Ala-streched0.pdb') # load the configuration

viewer = sith_viewer(atoms)       # create the visualization object
v = viewer.viewer
v                                 # generate the image

click on one of the atoms and then read the information of that atom

In [4]:
viewer.picked()

{'atom1': {'index': 16,
  'residueIndex': 0,
  'resname': 'MOL',
  'x': 5.116000175476074,
  'y': 2.0910000801086426,
  'z': 0.7170000076293945,
  'element': 'N',
  'chainname': 'A',
  'resno': 1,
  'serial': 17,
  'vdw': 1.55,
  'covalent': 0.71,
  'hetero': 0,
  'bfactor': 0,
  'altloc': '',
  'atomname': 'N',
  'modelIndex': 0,
  'name': '[MOL]1:A.N'},
 'component': 0}

## Bonds

Create a red bond between atoms 19-18 and 18-20.

In [5]:
viewer.add_bond(19, 18, [1, 0, 0], 0.05)
viewer.add_bond(20, 18, [1, 0, 0], 0.05);

Also, create a blue path of bonds with the next set of atoms 3, 0, 4, 6  (in that order), with bonds thicker than the previus example.

In [7]:
viewer.add_bonds([3, 0, 4], [0, 4, 6], [0, 0, 1], 0.1);

You can remove an specific bond, all bonds connected to one atom, a list of bonds or all the bonds in the structure. In that order, it would be:

In [ ]:
viewer.remove_bond(18, 19);
viewer.remove_bonds([0]);
viewer.remove_bonds()

## Angle

It is possible to add an angle

In [8]:
viewer.add_angle(19, 18, 20, n=30);
viewer.add_angle(3, 0, 4, n=1);
viewer.add_angle(0, 4, 6, color=[0.5, 0.5, 0], n=20);

also removing it or all in the graph

In [ ]:
viewer.remove_angle(19, 18, 20);

In [ ]:
viewer.remove_all_angles()

# Dihedrals

simple angles and dihedrals are pretty similar

In [9]:
viewer.add_dihedral(3, 0, 4, 6, color=[0, 0, 0], n=10);

In [ ]:
viewer.remove_dihedral(3, 0, 4, 6);

In [ ]:
viewer.remove_all_dihedrals()